In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 100)

### Load Dataset

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[-0.2737, -0.0764],
         [-1.6223,  0.5356],
         [ 0.2662, -0.8996],
         [-1.0217,  1.5210],
         [ 0.2009,  0.9339],
         [ 0.2648, -0.8644],
         [ 0.4594,  0.5204],
         [ 1.1733, -0.1749],
         [ 1.1813, -0.8230],
         [ 1.2561, -0.6371]]),
 tensor([[ 3.9151],
         [-0.8650],
         [ 7.7799],
         [-3.0034],
         [ 1.4205],
         [ 7.6753],
         [ 3.3603],
         [ 7.1291],
         [ 9.3724],
         [ 8.8834]])]

### Model

In [4]:
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

In [5]:
# initialize weight
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [6]:
# loss function
loss = nn.HuberLoss()

In [7]:
# optimizer
trainer = torch.optim.SGD(net.parameters(), lr=3e-1)

### Training

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 2.281389
epoch 2, loss 0.292488
epoch 3, loss 0.000197


In [9]:
net[0].bias.detach()

tensor([4.1901])

### Plot the estimation error

In [11]:
weights = []
bias = []
num_datas = [5 * 2 ** i for i in range(10)]
for n in num_datas:
    print(f'num_data : {n}')
    features, labels = d2l.synthetic_data(true_w, true_b, n)
    batch_size = 10
    data_iter = load_array((features, labels), batch_size)  

    net = nn.Sequential(nn.Linear(2,1))
    # initialize weight
    net[0].weight.data.normal_(0, 0.01)
    net[0].bias.data.fill_(0)
    trainer = torch.optim.SGD(net.parameters(), lr=3e-1)

    num_epochs = 3
    for epoch in range(num_epochs):
        for X, y in data_iter:
            l = loss(net(X), y)
            trainer.zero_grad()
            l.backward()
            trainer.step()
        l = loss(net(features), labels)
        print(f'epoch {epoch + 1}, loss {l:f}')
    
    weights.append(net[0].weight.detach())
    bias.append(net[0].bias.detach())

num_data : 5
epoch 1, loss 5.130519
epoch 2, loss 4.744869
epoch 3, loss 4.359220
num_data : 10
epoch 1, loss 3.861867
epoch 2, loss 3.650216
epoch 3, loss 3.444724
num_data : 20
epoch 1, loss 4.561119
epoch 2, loss 3.942480
epoch 3, loss 3.326511
num_data : 40
epoch 1, loss 3.703588
epoch 2, loss 2.699467
epoch 3, loss 1.764242
num_data : 80
epoch 1, loss 2.550804
epoch 2, loss 1.140921
epoch 3, loss 0.098840
num_data : 160
epoch 1, loss 0.733838
epoch 2, loss 0.000062
epoch 3, loss 0.000056
num_data : 320
epoch 1, loss 0.000673
epoch 2, loss 0.000044
epoch 3, loss 0.000045
num_data : 640
epoch 1, loss 0.000059
epoch 2, loss 0.000055
epoch 3, loss 0.000058
num_data : 1280
epoch 1, loss 0.000055
epoch 2, loss 0.000051
epoch 3, loss 0.000051
num_data : 2560
epoch 1, loss 0.000053
epoch 2, loss 0.000049
epoch 3, loss 0.000059


In [21]:
weights = [weight.squeeze() for weight in weights]
weights

[tensor([ 0.4359, -0.2370]),
 tensor([ 0.0463, -0.3339]),
 tensor([ 1.0093, -0.8524]),
 tensor([ 0.5241, -2.0940]),
 tensor([ 1.6429, -3.2081]),
 tensor([ 2.0007, -3.4000]),
 tensor([ 2.0021, -3.4006]),
 tensor([ 1.9992, -3.3980]),
 tensor([ 2.0015, -3.3997]),
 tensor([ 1.9963, -3.4023])]

In [28]:
diff_w = [weight - true_w for weight in weights]
diff_w

[tensor([-1.5641,  3.1630]),
 tensor([-1.9537,  3.0661]),
 tensor([-0.9907,  2.5476]),
 tensor([-1.4759,  1.3060]),
 tensor([-0.3571,  0.1919]),
 tensor([6.5470e-04, 3.3140e-05]),
 tensor([ 0.0021, -0.0006]),
 tensor([-0.0008,  0.0020]),
 tensor([0.0015, 0.0003]),
 tensor([-0.0037, -0.0023])]

In [23]:
true_w, true_b

(tensor([ 2.0000, -3.4000]), 4.2)

In [17]:
from matplotlib.pyplot import plot

weights_tensor = torch.as_tensor(weights)

ValueError: only one element tensors can be converted to Python scalars